# Final Capstone Report

## by Daniel da Cruz

### (Side note: Due to Foursquare API not functioning. The project has been adapted to compensate for the server-side error. The project below is focused on extracting data, preparing data, analyzing the data as well as providing a conclusion from the data.) :)

# The Business Problem 

Mr. Ronald is a 56 year old restaurenteer. He has been involved in the restaurant business since the age of 16. He has been quite succesful, and has built two businesses in the small town of Dundas, Ontario. 

His restaurants are part of a franchise called, "Ronald's Express." Mr. Ronald is the owner and founder of the franchise. Despite not having techincal background, he has identified a factors he believes has made his business a success. The menu is entirely designed by him. From all the years in the industry, his recipes have been created and refined to make his food the most delicious in his town. 

He sells burgers, tacos and pizzas using his very best recipes and cooking style. It is a town favourite. He know believes that Ronald's Express is ready to expand its business into Toronto, Ontario. The franchise is entirely family-owned, and he would like to pass it down to his sons when he retires. This will allow him to grow the business for many years to come. 

One of the key factors is opening a restaurant is the location. After a few failed busineses prior to Ronald's Expres, he realized that the location should have "plenty of feet." He is implying that the business need to be placed in a position to encounter a lot of people. He has also identified that his main customers are middle class people. He noticed that most of his customers are not exaclty "well off."

Ronald has been to Toronto a couple of times but its quite a big place. He has no idea where to begin looking for a place. He was told by his sons that he main need to find a data scientist to help with his problem. He has contacted me for assistance.

In this data science project, the target audience is Mr. Ronald and his family as they try to expand their family business into the Great Toronto Area. 

# The Data and Methodology

Due to Toronto being a world-class city, it keeps all census data on its website www.toronto.ca. This website contains all the data required to understand the city, and how it functions. The most recent census data will be used as part of the analysis. 

The neighborhood data for Toronto is available as an Excel spreadsheet and all unnecessary data was removed. The data was then adjusted so that it could be used for analyzing. The three most important coloumns of data, with respect to the project, were the original population size of the Toronto neighborhoods, the area size of the neighborhood and the average family income. From the data, will create a new column that will give us the population density. 

We created the population density data by taking the population size and dividing it by the area size. This is justifiable deducation and will tell us how many people are present per square kilometre. This is what Mr. Ronald was talking about without being able to fully explain what he meant. We can report this figure to him, and provide a recommendation for where to start his search. 

We will then focus on the average family income. Mr. Ronald had explained that both the lower class and upper class economic brackets did not seem to make up his customer base. He beleives that the middle can both afford his products, and also enjoy it most. These are rough estimations on his parts and we will like to go by his word. 

After doing a statistical analysis, we will be able to report the mean and median incomes for the city of Toronto. This will give us a reference point for finding the middle class. It will also give us information on which towns to avoid. Financial metrics tend to have skewed data and we will likely have to use a median average to get a good reference for the middle class. 

After extracting and cleaning data, we will do a statistical analysis and report the findings using Plotly. We will plot the data on graphs depending on how will be best present the findings. 


# Libraries Imported

In [18]:
import numpy as np # library to handle data in a vectorized manner
from bs4 import BeautifulSoup
import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!pip install geopy
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!pip install folium
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


## Toronto Location Data 

In [19]:
address = 'Toronto, Canada'

geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

43.6534817 -79.3839347


## Retrieving Census Data from CSV File
- The data has been cleaned and formatted for easier data handling.
- Density column has been added by dividing 'Total Population' by 'Total Area'

In [20]:
dataset = pd.read_csv(r"C:\Users\Daniel\Desktop\wellbeing-toronto-demographics.csv", delimiter=";")
dataframe = pd.DataFrame(dataset)
dataframe.head()

,Neighbourhood,AREA_S_CD,Total Area,Total Population,Density,Average Family Income,Pop 0 - 4 years,Pop 5 - 9 years,Pop 10 - 14 years,Pop 15 -19 years,Pop 20 - 24 years,Pop 25 - 29 years,Pop 30 - 34 years,Pop 35 - 39 years,Pop 40 - 44 years,Pop 45 - 49 years,Pop 50 - 54 years,Pop 55 - 59 years,Pop 60 - 64 years,Pop 65 - 69 years,Pop 70 - 74 years,Pop 75 - 79 years,Pop 80 - 84 years,Pop 85 years and over,Pop 6-12 years
0,West Humber-Clairville,1,"30,09",34100,1133,67240,2005,2135,2325,2180,2565,2465,2400,2440,2595,2375,1955,1800,1415,1150,1015,715,465,305,3126
1,Mount Olive-Silverstone-Jamestown,2,"4,6",32790,7128,52745,2680,2680,2685,2285,2410,2590,2675,2605,2450,2130,1700,1495,1200,910,775,500,270,145,3782
2,Thistletown-Beaumond Heights,3,"3,4",10140,2982,71300,615,625,645,630,655,650,650,730,790,735,590,520,420,400,425,385,255,190,893
3,Rexdale-Kipling,4,"2,5",10485,4194,65215,580,645,665,640,630,600,705,815,840,810,720,600,505,435,460,435,345,285,919
4,Elms-Old Rexdale,5,"2,9",9550,3293,56515,725,700,745,655,615,645,640,735,740,750,585,505,385,310,275,205,150,55,1020


# Obtaining Location Coordinates of Toronto Neighborhoods

In [21]:
neighborhood_lat = []
neighborhood_long = []
for i in dataframe['Neighbourhood']: 
    address = '{}, Toronto, Canada'.format(i)
    geolocator = Nominatim(user_agent="foursquare_agent")
    location = geolocator.geocode(address)
    if location == None:
        neighborhood_lat.append('unknown')
        neighborhood_long.append('unknown')
    else:
        latitude = location.latitude
        longitude = location.longitude
        neighborhood_lat.append(latitude)
        neighborhood_long.append(longitude)

## Adding the Latitude and Longitude Coordinates to the Dataframe

In [22]:
dataframe.insert(6, 'Latitude', neighborhood_lat)
dataframe.insert(7, 'Longitude', neighborhood_long)
dataframe.head(15)

,Neighbourhood,AREA_S_CD,Total Area,Total Population,Density,Average Family Income,Latitude,Longitude,Pop 0 - 4 years,Pop 5 - 9 years,Pop 10 - 14 years,Pop 15 -19 years,Pop 20 - 24 years,Pop 25 - 29 years,Pop 30 - 34 years,Pop 35 - 39 years,Pop 40 - 44 years,Pop 45 - 49 years,Pop 50 - 54 years,Pop 55 - 59 years,Pop 60 - 64 years,Pop 65 - 69 years,Pop 70 - 74 years,Pop 75 - 79 years,Pop 80 - 84 years,Pop 85 years and over,Pop 6-12 years
0,West Humber-Clairville,1,"30,09",34100,1133,67240,43.722563,-79.597039,2005,2135,2325,2180,2565,2465,2400,2440,2595,2375,1955,1800,1415,1150,1015,715,465,305,3126
1,Mount Olive-Silverstone-Jamestown,2,"4,6",32790,7128,52745,unknown,unknown,2680,2680,2685,2285,2410,2590,2675,2605,2450,2130,1700,1495,1200,910,775,500,270,145,3782
2,Thistletown-Beaumond Heights,3,"3,4",10140,2982,71300,unknown,unknown,615,625,645,630,655,650,650,730,790,735,590,520,420,400,425,385,255,190,893
3,Rexdale-Kipling,4,"2,5",10485,4194,65215,43.722114,-79.572292,580,645,665,640,630,600,705,815,840,810,720,600,505,435,460,435,345,285,919
4,Elms-Old Rexdale,5,"2,9",9550,3293,56515,43.72177,-79.552173,725,700,745,655,615,645,640,735,740,750,585,505,385,310,275,205,150,55,1020
5,Kingsview Village-The Westway,6,"5,1",21725,4260,60440,unknown,unknown,1500,1445,1360,1300,1445,1405,1490,1510,1635,1515,1345,1205,870,850,875,785,555,290,1976
6,Willowridge-Martingrove-Richview,7,"5,5",21345,3881,70855,unknown,unknown,1035,1140,1205,1325,1265,1080,1145,1375,1555,1610,1380,1225,950,1015,1160,1075,865,500,1667
7,Humber Heights-Westmount,8,"2,8",10580,3779,69750,43.695785,-79.520832,510,500,595,540,505,470,590,690,790,720,620,595,450,505,585,620,710,545,778
8,Edenbridge-Humber Valley,9,"5,5",14945,2717,142550,43.670672,-79.518855,570,720,875,1005,975,835,735,940,1185,1170,1160,985,720,635,635,550,415,320,1099
9,Princess-Rosethorn,10,"5,2",11200,2154,152850,unknown,unknown,380,580,850,905,710,420,290,545,945,1030,965,785,560,420,445,460,420,235,1005


Note: The location data for some neighborhoods cannot be found. This poses a problem. Instead will continue to use the available data. Therefore, some neighborhoods will not show up on the map. 

## Mapping the neighborhoods of Toronto

In [23]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10.4)

for lat, lng, neighborhood in zip(dataframe['Latitude'], dataframe['Longitude'], dataframe['Neighbourhood']):
    if lat and lng == 'unknown':
        continue
    else:
        label ='{}'.format(neighborhood)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color='blue',
            fill=True,
            fill_color='#3186cc',
            fill_opacity=0.7,
            parse_html=False).add_to(map_toronto)
map_toronto